# Import Dependencies

In [32]:
import csv
import os
import pandas as pd

# Import Data & Create the Data Frame

In [33]:
# Load data
data_to_clean = 'data/29282-0001-Data.tsv'
data = pd.read_csv(data_to_clean, sep='\t', low_memory=False)

# Column specifications
Ids = ['M2ID']

input_columns_to_select = {
    'age': ['B1PAGE_M2'],  # single column
    'sex': ['B1PRSEX'], # single column
    'caffeine': ['B4AD64', 'B4AD74', 'B4AD14', 'B4AD24', 'B4AD34', 'B4AD44', 'B4AD54'], # Sunday to Saturday
    'napped': ['B4AD63', 'B4AD73', 'B4AD13', 'B4AD23', 'B4AD33', 'B4AD43', 'B4AD53'], # Sunday to Saturday
    'nap_length': ['B4AD63S', 'B4AD73S', 'B4AD13S', 'B4AD23S', 'B4AD33S', 'B4AD43S', 'B4AD53S'], # Sunday to Saturday
    'sleep_medication': ['B4AD57', 'B4AD67', 'B4AD77', 'B4AD17', 'B4AD27', 'B4AD37', 'B4AD47'], # Sunday to Saturday
    'alcohol': ['B4AD65', 'B4AD75', 'B4AD15', 'B4AD25', 'B4AD35', 'B4AD45', 'B4AD55'], # Sunday to Saturday
    'exercise': ['B4AD62', 'B4AD72', 'B4AD12', 'B4AD22', 'B4AD32', 'B4AD42', 'B4AD52'], # Sunday to Saturday
    'noise': ['B4AD612', 'B4AD712', 'B4AD112', 'B4AD212', 'B4AD312', 'B4AD412', 'B4AD512'], # Sunday to Saturday
    'bedtime': ['B4AD58', 'B4AD68', 'B4AD78', 'B4AD18', 'B4AD28', 'B4AD38', 'B4AD48'], # Sunday to Saturday    
    'wakeup_time': ['B4AD515', 'B4AD615', 'B4AD715', 'B4AD115','B4AD215', 'B4AD315', 'B4AD415'] # Sunday to Saturday
}

output_columns_to_select = {
    'quality': ['B4AD620', 'B4AD720', 'B4AD120', 'B4AD220', 'B4AD320', 'B4AD420', 'B4AD520'], # Monday to Sunday
    'rested': ['B4AD618', 'B4AD718', 'B4AD118', 'B4AD218', 'B4AD318', 'B4AD418', 'B4AD518'], # Monday to Sunday
    'minutes_to_sleep': ['B4AD69', 'B4AD79', 'B4AD19', 'B4AD29', 'B4AD39', 'B4AD49', 'B4AD59'], # Monday to Sunday
    'depth_of_sleep': ['B4AD617', 'B4AD717', 'B4AD117', 'B4AD217', 'B4AD317', 'B4AD417', 'B4AD517'], # Monday to Sunday
    'difficulty_of_sleep': ['B4AD610', 'B4AD710', 'B4AD110', 'B4AD210', 'B4AD310', 'B4AD410', 'B4AD510'] # Monday to Sunday
}

day_names = ["day1", "day2", "day3", "day4", "day5", "day6", "day7"]

# First, we will create a new DataFrame with one row per ID per day : Repeat each row 7 times, one for each day
df_expanded = data.loc[data.index.repeat(7)].reset_index(drop=True)
df_expanded['day'] = day_names * len(data)  # Assign day names

# Now we fill in the single-column variables (they don't depend on day)
for var_name, col_list in input_columns_to_select.items():
    if len(col_list) == 1:
        # Single column variable: just copy it over (its value repeats for all 7 days)
        df_expanded[var_name] = df_expanded[col_list[0]]

for var_name, col_list in output_columns_to_select.items():
    if len(col_list) == 1:
        # If any output columns were single-day (none in this example, but just in case)
        df_expanded[var_name] = df_expanded[col_list[0]]

# Now handle the multi-day variables : For each multi-day variable, we select the correct column based on the day
def assign_multiday_variable(df, var_name, columns):
    # Columns is a list of length 7 : We have a 'day' column that can tell us which day row corresponds to.
    for i, d in enumerate(day_names):
        day_mask = df['day'] == d
        df.loc[day_mask, var_name] = df.loc[day_mask, columns[i]]

for var_name, col_list in input_columns_to_select.items():
    if len(col_list) == 7:
        assign_multiday_variable(df_expanded, var_name, col_list)

for var_name, col_list in output_columns_to_select.items():
    if len(col_list) == 7:
        assign_multiday_variable(df_expanded, var_name, col_list)

final_df = df_expanded[[*Ids, 'day'] + list(input_columns_to_select.keys()) + list(output_columns_to_select.keys())]

def calculate_hours_slept(bedtime, wakeuptime):
    if bedtime < 1200:  # Bedtime after midnight
        return (wakeuptime - bedtime) / 100.0  # Convert to hours
    else:  # Bedtime before midnight
        return ((2400 - bedtime) + wakeuptime) / 100.0  # Convert to hours

# Apply the function to the DataFrame
final_df = final_df.copy()
final_df['hours_slept'] = final_df.apply(
    lambda row: calculate_hours_slept(row['bedtime'], row['wakeup_time']),
    axis=1
)


# Filter the Data

In [44]:
# Used for calculating the actual sleep time
def calculate_sleep_duration(bedtime_min, wakeup_min):
    """
    Calculates sleep duration in hours based on bedtime and wakeup time in HHMM format.
    Assumes bedtime and wakeup_time are in 24-hour format without any AM/PM distinction.
    """
    # Convert bedtime and wakeup_time to minutes since midnight
    bedtime_hours = bedtime_min // 100
    bedtime_minutes = bedtime_min % 100
    wakeup_hours = wakeup_min // 100
    wakeup_minutes = wakeup_min % 100
    
    bedtime_total = bedtime_hours * 60 + bedtime_minutes
    wakeup_total = wakeup_hours * 60 + wakeup_minutes
    
    if bedtime_total < wakeup_total:
        duration = (wakeup_total - bedtime_total) / 60  # Convert minutes to hours
    else:
        duration = ((1440 - bedtime_total) + wakeup_total) / 60  # Overnight sleep
    
    return duration

In [47]:
# Given mappings from your dictionaries to final_df columns
input_bounds = {
    "caffeine": (0, 12),
    "napped": (1, 2),
    "nap_length": (0, 996),
    "sleep_medication": (1, 2),
    "alcohol": (0, 12),
    "exercise": (0, 996),
    "noise": (1, 2),
    "bedtime": (0, 9996),
    "wakeup_time": (0, 9996)
}

output_bounds = {
    "quality": (1, 5),
    "rested": (1, 5),
    "minutes_to_sleep": (0, 996),
    "depth_of_sleep": (1, 5),
    "difficulty_of_sleep": (1, 5)
}

# Start with a mask of all True (meaning keep all rows initially)
mask = pd.Series(True, index=final_df.index)

# Check inputs against their bounds
for var, (lower, upper) in input_bounds.items():
    if var in final_df.columns:
        if var == "nap_length":
            # Special handling for napped column
            final_df["nap_length"] = final_df["nap_length"].apply(
                lambda x: 0 if pd.isna(x) or not (lower <= x <= upper) else x
            )
        else:
            mask = mask & (final_df[var].between(lower, upper, inclusive='both'))

# Check outputs against their bounds
for var, (lower, upper) in output_bounds.items():
    if var in final_df.columns:
        mask = mask & (final_df[var].between(lower, upper, inclusive='both'))

if final_df['day'].dtype == str:
    final_df['day'] = final_df['day'].str.replace('day', '').astype(int)

# Check if 'sleep_duration' exists and is correctly calculated
if 'sleep_duration' not in final_df.columns or final_df['sleep_duration'].isnull().sum() > 0:
    final_df['sleep_duration'] = final_df.apply(
        lambda row: calculate_sleep_duration(row['bedtime'], row['wakeup_time']),
        axis=1
    )
    # Impute any missing values with median
    final_df['sleep_duration'].fillna(final_df['sleep_duration'].median(), inplace=True)
    print("Calculated and imputed 'sleep_duration'.")
else:
    print("'sleep_duration' already exists and is properly calculated.")

# Drop 'hours_slept' if it's redundant
if 'hours_slept' in final_df.columns:
    final_df.drop(['hours_slept'], axis=1, inplace=True)
    print("Dropped 'hours_slept' as it's redundant with 'sleep_duration'.")

# Apply the mask to filter the DataFrame
final_df = final_df[mask].reset_index(drop=True)

'sleep_duration' already exists and is properly calculated.


# Writing to the actual file

In [48]:
# Specify the folder & File Name
output_folder = 'data'
output_filename = 'updated.tsv'
output_path = os.path.join(output_folder, output_filename)

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Save the DataFrame to a TSV file
final_df.to_csv(output_path, sep='\t', index=False)